In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=b0f2a4f0dfbed789b4ac173591456d1a0160adb368a623327cfba5c7650fc7c3
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import cv2
import numpy as np
from deepface import DeepFace
import os
import torch
import torch.nn as nn

def normalize_brightness_contrast_grayscale(image):
    # Normalize brightness using LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    lab = cv2.merge((l, a, b))
    normalized_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Contrast normalization using CLAHE
    lab = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    normalized_contrast_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Convert to grayscale
    gray_image = cv2.cvtColor(normalized_contrast_image, cv2.COLOR_BGR2GRAY)

    return gray_image


def get_all_embeddings(img_path):
    # embeddings = []
    # for img_path in img_paths:
    image = cv2.imread(img_path)

    # Normalize brightness, contrast, and convert to grayscale
    processed_image = normalize_brightness_contrast_grayscale(image)

    temp_path = "temp_grayscale.jpg"
    cv2.imwrite(temp_path, processed_image)

    embedding = DeepFace.represent(img_path=temp_path, model_name="Facenet512", detector_backend='retinaface')
    embarr = np.array(embedding[0]["embedding"])
    # embeddings.append(embarr)

    # Clean up the temporary file
    os.remove(temp_path)

    return np.array(embarr)


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=512, out_channels=128, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = x.unsqueeze(0).reshape(1,512,1)
        x = self.conv1d(x)
        return x

def get_CNN_embeddings(model, embed):

    # Convert the embedding to a PyTorch tensor if it's not already
    embed_tensor = torch.tensor(embed).float()

    # Add a batch dimension if necessary (assuming model expects batched input)
    embed_tensor = embed_tensor.unsqueeze(0)  # Add batch dimension

    # Pass the embedding through the model
    with torch.no_grad():  # No need to compute gradients
        output = model(embed_tensor)

    # Remove the batch dimension if necessary
    output = output.squeeze(0).squeeze(1)  # Remove batch dimension

    # Convert the output tensor back to a list or numpy array if needed
    output_list = output.numpy()  # or output.tolist()
    return output_list


def matching_percentage(img_path1, img_path2):

    # Create the model instance
    model = CNNModel()
    model.eval()

    img1_emb = get_all_embeddings(img_path1)
    img2_emb = get_all_embeddings(img_path2)

    vector1 = get_CNN_embeddings(model, img1_emb)
    vector2 = get_CNN_embeddings(model, img2_emb)

    # Calculate the number of matching elements
    matching_elements = np.sum(vector1.round(0) - vector2.round(0) == 0)

    # Calculate the total number of elements
    total_elements = vector1.shape[0]

    # Calculate the percentage of matching elements
    percentage_matching = (matching_elements / total_elements) * 100

    # Print the size of the embedded vectors
    embedded_size = vector1.shape
    # print(f"Size of the embedded vectors: {embedded_size}, {matching_elements}")

    # Print the percentage of matching elements
    print(f"Percentage of matching elements: {percentage_matching:.2f}%")

    return percentage_matching

# Assuming you have a directory with images organized in subfolders for each class
dir = '/content/drive/MyDrive/AK/A'
imgfolders = [os.path.join(dir, folder) for folder in os.listdir(dir)]

# embedding_path =

# List to store paths of all images
imgpaths_A = []
for folder in imgfolders:
    for path in os.listdir(folder):
        imgpaths_A.append(os.path.join(folder, path))

24-07-31 11:43:13 - Directory /root/.deepface created
24-07-31 11:43:13 - Directory /root/.deepface/weights created


In [5]:
for i in range(len(imgpaths_A[:-1])):
  matching_percentage(imgpaths_A[i], imgpaths_A[i+1])

24-07-31 11:43:33 - facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:01<00:00, 91.3MB/s]


24-07-31 11:43:40 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:00<00:00, 240MB/s] 


Percentage of matching elements: 89.84%
Percentage of matching elements: 82.03%
Percentage of matching elements: 91.41%
